In [1]:
!pip install spacy_transformers
!pip install -U spacy
!python -m space download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.7/191.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 47.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.1
    Uninstalling spacy-3.5.1:
      Successfully uninstalled spacy-3.5.1
/opt/conda/bin/python: No module named space


In [2]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm  
import json

In [3]:
cv_data = json.load(open('/kaggle/input/resume-training/train_data_new.json'))

In [4]:
cv_data = cv_data[:35]

In [5]:
!python -m spacy init fill-config /kaggle/input/resumeparsing/data/training/base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [6]:
def get_spacy_doc(data):
    nlp = spacy.load('en_core_web_sm')
    db = DocBin()
        
    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        annot = annot['entities']
    
        ents = []
        entity_indices = []
            
        for start, end, label in annot:
            skip_entity = False
            for idx in range(start, end):
                if idx in entity_indices:
                    skip_entity = True
                    break
            if skip_entity == True:
                continue 
                    
            entity_indices = entity_indices + list(range(start, end))  
                
            try:
                span = doc.char_span(start, end, label = label, alignment_mode = 'strict')
            except:
                continue
                
            if span is not None:
                ents.append(span)

        try:
            doc.ents = ents
            db.add(doc)
        except:
            pass 
        
    return db

In [10]:
train = cv_data[:25]
test = cv_data[25:]

db = get_spacy_doc(train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(test)
db.to_disk('test_data.spacy')

100%|██████████| 10/10 [00:00<00:00, 154.78it/s]


In [11]:
!python -m spacy train /kaggle/working/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-05-08 18:13:26,742] [INFO] Set up nlp object from config
[2023-05-08 18:13:26,756] [INFO] Pipeline: ['transformer', 'ner']
[2023-05-08 18:13:26,760] [INFO] Created vocabulary
[2023-05-08 18:13:26,762] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a 

In [12]:
nlp = spacy.load('/kaggle/working/output/model-best')

In [13]:
txt = 'Hello! My name is Anushtha Bageria. I am a software developer and UI/UX designer. Currently working as SWE Intern in Microsoft. and my email is https://www.indeed.com/code/anushthabageria/nlp-course-project/edit'

In [14]:
doc = nlp(txt)
for ent in doc.ents:
    print(ent.label_.upper(),"  =>  ", ent.text)

NAME   =>   Anushtha Bageria
JOB   =>   software developer
JOB   =>   UI/UX designer


In [15]:
from spacy import displacy
displacy.render(doc, style='ent', jupyter=True)

In [16]:
with open('/kaggle/input/resumeparsing/data/test/Alice Clark CV.txt', 'r', encoding = 'utf-8') as file:
    test_data = file.read()
test_data

'Alice Clark \nAI / Machine Learning \n \nDelhi, India Email me on Indeed \n• \n20+ years of experience in data handling, design, and development \n• \nData Warehouse: Data analysis, star/snow flake scema data modelling and design specific to \ndata warehousing and business intelligence \n• \nDatabase: Experience in database designing, scalability, back-up and recovery, writing and \noptimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes. \nCloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure, \nStream Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake \nanalytics(U-SQL) \nWilling to relocate anywhere \n \nWORK EXPERIENCE \nSoftware Engineer \nMicrosoft – Bangalore, Karnataka \nJanuary 2000 to Present \n1. Microsoft Rewards Live dashboards: \nDescription: - Microsoft rewards is loyalty program that rewards Users for browsing and shopping \nonline. Microsoft Rewards members can earn points when 

In [17]:
import re

def cleanResume(resumeText):
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    return resumeText

test_data = cleanResume(test_data)
test_data

'Alice Clark AI Machine Learning Delhi India Email me on Indeed 20 years of experience in data handling design and development Data Warehouse Data analysis star snow flake scema data modelling and design specific to data warehousing and business intelligence Database Experience in database designing scalability back up and recovery writing and optimizing SQL code and Stored Procedures creating functions views triggers and indexes Cloud platform Worked on Microsoft Azure cloud services like Document DB SQL Azure Stream Analytics Event hub Power BI Web Job Web App Power BI Azure data lake analytics U SQL Willing to relocate anywhere WORK EXPERIENCE Software Engineer Microsoft Bangalore Karnataka January 2000 to Present 1 Microsoft Rewards Live dashboards Description Microsoft rewards is loyalty program that rewards Users for browsing and shopping online Microsoft Rewards members can earn points when searching with Bing browsing with Microsoft Edge and making purchases at the Xbox Store t

In [18]:
doc = nlp(test_data)
displacy.render(doc, style='ent', jupyter=True)